In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark import SparkConf
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

# from src.serial import NaiveDBSCAN, MatrixDBSCAN
# from src.utils import DataLoader, Evaluation, timeit
# from src.settings import UNKNOWN, NOISE

import numpy as np

# broadcast variable
b_dataset = None
b_eps = None
b_min_pts = None


def load_data_label(path):
    pts = sc.textFile(path).map(lambda x: x.strip().split()[:-1]).map(lambda x: tuple([float(i) for i in x]))
    return pts.collect()

def load_data(path):
    pts = sc.textFile(path).map(lambda x: x.strip().split()).map(lambda x: tuple([float(i) for i in x]))
    return pts.collect()

def _bounds_coordinates(bin_bounds):

    lower_cdnts = [[low] for low in  bin_bounds[0][:-1]]
    upper_cdnts = [[high] for high in bin_bounds[0][1:]]
    
    # super stupid implementation, optimization needed
    for bound in bin_bounds[1:]:
        lower_tmp = []
        upper_tmp = []
        
        for bc in bound[:-1]:
            lower_tmp.extend([lc + [bc] for lc in lower_cdnts])
        lower_cdnts = lower_tmp
        
        for bc in bound[1:]:
            upper_tmp.extend([uc + [bc] for uc in upper_cdnts])
        upper_cdnts = upper_tmp
        
    return np.array(lower_cdnts), np.array(upper_cdnts)

# @timeit
def spatial_partition(dataset, par_each_dim, eps):
    """
    Partitions the given dataset into spatial bins based on the specified parameters.

    Args:
        dataset (numpy.ndarray): The input dataset to be partitioned.
        par_each_dim (tuple): A tuple specifying the number of partitions in each dimension.
        eps (float): A value representing the maximum distance between two samples.

    Returns:
        pyspark.rdd.RDD: An RDD containing the indexed data, where each element is a tuple
        of the partition ID and a list of point IDs belonging to that partition.
    """
    tp_par = par_each_dim
    num_par = np.prod(par_each_dim)
    
    # cut bins
    bounds_dim = np.concatenate(([np.min(dataset, axis=0)], [np.max(dataset, axis=0)]), axis=0).T   # (d, 2)
    
    bin_bounds = []
    for i in range(len(tp_par)):
        dim_bins = np.linspace(*bounds_dim[i], tp_par[i]+1, endpoint=True)
        bin_bounds.append(dim_bins)
    
    lower_bounds, upper_bounds = _bounds_coordinates(bin_bounds)
    lower_bounds -= eps
    upper_bounds += eps
    if np.min(upper_bounds-lower_bounds) < 2*eps:
        raise Warning('Partitions Overlap too much')

    # scatter points into bins with eps
    indexed_data = []
    # double loop to ensure border points could be given multiple partition ID
    for id_pts in range(len(dataset)):     # index of point in dataset
        for id_ptt in range(num_par):
            if not (dataset[id_pts] > lower_bounds[id_ptt]).all():
                continue
            if not (dataset[id_pts] < upper_bounds[id_ptt]).all():
                continue
            indexed_data.append([id_ptt, id_pts])
            
    res = sc.parallelize(indexed_data).groupByKey().map(lambda x: [x[0], list(x[1])])
    return res

# def local_dbscan(partioned_rdd, method='matrix', metric='euclidian'):

#     dataset = np.array([b_dataset.value[idp] for idp in partioned_rdd])
#     if method == 'matrix':
#         dbscan_obj = MatrixDBSCAN(dataset, b_eps.value, b_min_pts.value, metric) 
#     else:
#         dbscan_obj = NaiveDBSCAN(dataset, b_eps.value, b_min_pts.value, metric) 
#     dbscan_obj.predict()
#     is_core_list = dbscan_obj._find_core_pts()
    
#     return list(zip(zip(partioned_rdd, is_core_list), dbscan_obj.tags))

# @timeit
# def merge(local_tags, dataset):
#     global_tags = [UNKNOWN] * len(dataset)
#     is_tagged = [0] * len(dataset)
#     last_max_label = 0
#     for local in local_tags:
#         np_local = np.array(local[-1])
#         np_local[:, -1] += last_max_label

#         last_max_label = np.max(np_local[:, -1])
        
#         # check and merge overlapped points
#         tagged_indices = np.nonzero(is_tagged)[0]
#         for tmp_i in range(len(np_local)):
#             # should do tag check
#             (p_id, is_core), label = np_local[tmp_i]
#             if p_id in tagged_indices and is_core==1:
#                 np_local[-1][np_local[-1]==label] = global_tags[p_id]
        
#         # update global tags
#         for (p_id, is_core), label in np_local:
#             if is_tagged[p_id]==1:
#                 continue
#             global_tags[p_id] = label
#             is_tagged[p_id] = 1
#     return global_tags


24/05/14 17:19:09 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:502)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:486)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnection.waitForCommands(ClientServerConnection.ja

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.NoClassDefFoundError: Could not initialize class org.apache.spark.unsafe.array.ByteArrayMethods
	at org.apache.spark.memory.MemoryManager.defaultPageSizeBytes$lzycompute(MemoryManager.scala:264)
	at org.apache.spark.memory.MemoryManager.defaultPageSizeBytes(MemoryManager.scala:254)
	at org.apache.spark.memory.MemoryManager.$anonfun$pageSizeBytes$1(MemoryManager.scala:273)
	at scala.runtime.java8.JFunction0$mcJ$sp.apply(JFunction0$mcJ$sp.java:23)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.memory.MemoryManager.<init>(MemoryManager.scala:273)
	at org.apache.spark.memory.UnifiedMemoryManager.<init>(UnifiedMemoryManager.scala:58)
	at org.apache.spark.memory.UnifiedMemoryManager$.apply(UnifiedMemoryManager.scala:207)
	at org.apache.spark.SparkEnv$.create(SparkEnv.scala:320)
	at org.apache.spark.SparkEnv$.createDriverEnv(SparkEnv.scala:194)
	at org.apache.spark.Sp